<img src="files/logo.jpg"/>

## はじめに

このOptunaの基本的な使い方に関するハンズオンは、Google Colaboratry で書かれたノートブックに沿って進めます。Google Colaboratry ではブラウザ上で Python プログラムを実行することができます。

試しに以下のコードを実行してみましょう。

In [ ]:
print('Hello Notebook!')  # shift + enter で実行できます

このノートブックで使うモジュールをインポートしておきましょう。



In [ ]:
!pip install optuna

import optuna
import sklearn
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics

## 機械学習の復習

このハンズオンでは機械学習の基礎的な知識が必要となります。
少なくとも次のキーワードを復習した上で読むようにしてください。

*   ***モデル (model)***
*   ***パラメタ (parameter)***
*   ***ハイパーパラメータ (hyperparameter)***
*   ***線形回帰 (regression)***
*   ***正則化 (regularization)***
*   ***精度 (accuracy)***
*   ***訓練集合 (training set)***
*   ***評価集合 (validation set)***
*   ***scikit-learn***




例題として、住宅価格を予測する機械学習モデルを学習・評価してみましょう。
Boston データセットにはボストン市内における 150 地域のデータが含まれており、各地域について不動産税率や平均住宅価格など 14 項目の数値が記録されています。そこで、13 個の入力変数から地域の平均住宅価格を予測する線形回帰モデルを学習させます。

scikit-learn を使うと、上記の処理を簡潔に書くことができます。

In [ ]:
# ボストン物件価格のデータセットをロードする
#     X: 各行が地域に、各列がそれぞれの入力変数に対応 (506 * 13 の行列)
#     y: 各個体のラベル (506 要素のベクトル)
X, y = sklearn.datasets.load_boston(return_X_y=True)

# データセットを訓練用と評価用に分割する
#     X_train, y_train: 訓練用データセット
#     X_val, y_val: 評価用データセット
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)

# 線形回帰オブジェクトの初期化
#     L1 正則化 (Lasso) を適用
#     alpha は正則化の強さに対応するハイパーパラメータ
alpha = 1.0
model = sklearn.linear_model.Lasso(alpha=alpha)

# 訓練用データセットで学習する
model.fit(X_train, y_train)

# 評価用データセットの出力を予測する
y_pred = model.predict(X_val)

# モデルの性能を評価する
error = sklearn.metrics.mean_squared_error(y_val, y_pred)

print(error)

## 目的関数を書く

まずは簡単な二次関数の最小化問題を考えてみましょう。
$(x - 2) ^ 2$ が最小となる $x$ を求める問題です。
$(x - 2) ^ 2$ のような最小化対象の関数を**目的関数** ***(objective function)*** と呼びます。
以下のコードを実行して目的関数を定義してみましょう。

In [ ]:
def objective(x):
    return (x - 2) ** 2

人間は $x = 2$ が答えであると解りますが、Optuna はこの問題の解き方を知りません。
そこで実際に複数の $x$ を代入して出力を計算し、出力が最小となった $x$ が最も良い解であると判断します。
以下のような計算をするイメージです。

In [ ]:
import random

outputs = []  # 計算結果を保存しておくリスト

# 区間　[-100, 100]　から適当な x を 20 パターン試す
for _ in range(20):
    x = random.uniform(-100, 100)
    objective_value = objective(x)
    outputs.append(objective_value)

minimum_output = min(outputs)
print('目的関数の最小値: ' + str(minimum_output))

上記の例ではランダムな $x$ を 20 個試していますが、Optuna はこれまでに試した入力 $x$ と出力 $(x - 2) ^ 2$ からヒントを得ることで、より良い結果が得られそうな $x$ に当たりをつけていきます。

## Optuna を使って目的関数を最小化する

Optuna を使って $(x - 2) ^ 2$ を最小化する $x$ を求めてみましょう。
以下の 4 つのステップが必要となります。

1.   目的関数を定義する
2.   目的関数の内部で適当な変数を決める ***(suggest)***
3.   実験 ***study*** オブジェクトを作成する。
4.   施行 ***(trial)*** の回数を設定して最適化を開始する ***(optimize)***

目的関数を定義します。
先ほど定義した目的関数とは少し違っており、$x$ の代わりに ***trial*** オブジェクトを引数としています。
これは Optuna で目的関数を書くときの決まりごとです。
***trial*** オブジェクトの ***suggest_float*** メソッド実行したタイミングで、次に試すべき $x$ の値が提案 ***(suggest)*** されます。
提案された $x$ を使って関数からの出力 $(x - 2) ^ 2$ を計算します。

In [ ]:
def objective(trial):                                           # 目的関数の定義
    x = trial.suggest_float("x", -100, 100)  # 区間　[-100, 100]　から適当な x を決める
    return (x - 2) ** 2                                          # 目的関数の計算結果を返す

この目的関数を最小化する $x$ を求めることが Optuna の役目です。
以下のコードを実行してみましょう。
実験 ***(study)*** のオブジェクトを作成し、***optimize*** メソッドに目的関数と施行の回数を与えることで最小化実験が始まります。

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=20)

20 行のログが出力されているはずです。
Optuna が $(x - 2) ^ 2$ を 20 回実行し、そのつど異なる $x$ を試したことを意味しています。

実験結果を見てみましょう。
***study.best_value*** で 20 試行中の最小となる出力 $(x - 2) ^ 2$ の値が、***study.best_params*** でその時の入力 $x$ がわかります。

In [ ]:
print("目的関数の最小値: " + str(study.best_value))
print("出力が最小となる入力: " + str(study.best_params))

## Optuna で機械学習のハイパーパラメータを決める

以下のコードでは、Lasso 回帰を使って、ボストンにおける地域毎の平均住宅価格を予測しています。
正則化の強さは Lasso 回帰にとって重要なハイパーパラメータですが、この値を人手で適切に決めるには手間がかかります。
そこで Optuna を使って自動チューニングするようコードを改変します。
Optunaはplatform agnosticなので、もちろんscikit-learnのコードにも適用する事ができます。


In [ ]:
# ハイパーパラメータの決定
alpha = 1.0

# モデルの学習と評価
X, y = sklearn.datasets.load_boston(return_X_y=True)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)
model = sklearn.linear_model.Lasso(alpha=alpha)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
error = sklearn.metrics.mean_squared_error(y_val, y_pred)

# 評価性能の出力
print(error)

Optuna は目的関数からの出力を最小化 (または最大化) するような入力値を求めるツールでした。
一方で、上記の機械学習の学習・評価ロジックは、ハイパーパラメータを入力、評価性能を出力とする目的関数ととらえることができます。
したがって学習・評価ロジックをそのまま目的関数としてラップすることで、Optuna の形式に従ったコードになります。

In [ ]:
def objective(trial):
    # ハイパーパラメータの決定
    alpha = trial.suggest_float("alpha", 0.0, 2.0)
  
    # モデルの学習と評価
    X, y = sklearn.datasets.load_boston(return_X_y=True)
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)
    model = sklearn.linear_model.Lasso(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)
  
    # 平均二乗誤差を目的関数からの出力とする
    return error

ハイパーパラメータを自動探索してみましょう。
20 回の学習・評価が実行されたのち、最も良かった出力と、その時のハイパーパラメータが表示されます。

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=20)

print("最良の誤差: " + str(study.best_value))
print("最良のハイパーパラメータ: " + str(study.best_params))

## 条件付きハイパーパラメータを探索する

正則化の強さに加えて『どの種類の正則化を使うか (Ridge か Lasso か)』も探索したいとします。
探索したいハイパーパラメータは以下の 3 個となります。

- `regression_method`: `'ridge'` か `'lasso'` のいずれか
- `ridge_alpha`: `ridge` における正則化の強さ
- `lasso_alpha`: `lasso` における正則化の強さ

`ridge` を使う場合には `ridge_alpha` のみを、`lasso` を使う場合には `lasso_alpha` のみを探索したいことに注意してください。
つまり、探索対象のハイパーパラメータが `regression_method` の値によって変化します。
Optuna では、このような条件つきの探索空間を通常のPythonのコードを書くようにして扱うことができます。
このように通常のPythonコードを書くように探索空間が記述できることから、
Optunaの探索空間は"Pythonic search spaces"であると言われます。

In [ ]:
def objective(trial):
    #  データのロードと分割
    X, y = sklearn.datasets.load_boston(return_X_y=True)
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)

    #  ハイパーパラメータの決定とモデルオブジェクトの初期化
    regression_method = trial.suggest_categorical("regression_method", ('ridge', 'lasso'))
    if regression_method == 'ridge':
        ridge_alpha = trial.suggest_float("ridge_alpha", 0.0, 2.0)
        model = sklearn.linear_model.Ridge(alpha=ridge_alpha)
    else:
        lasso_alpha = trial.suggest_float("lasso_alpha", 0.0, 2.0)
        model = sklearn.linear_model.Lasso(alpha=lasso_alpha)
        
    # モデルの学習と評価
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)
  
    # 平均二乗誤差を目的関数からの出力とする
    return error

study = optuna.create_study()
study.optimize(objective, n_trials=20)

print("最良の誤差: " + str(study.best_value))
print("最良のハイパーパラメータ: " + str(study.best_params))

## おわりに

以上でOptunaの基本的な使い方に関するハンズオンは終わりです。
今回はscikit-learnを題材にOptunaの使い方を説明しましたが、Optunaはplatform agnosticなライブラリなため、他のどんなライブラリとも組み合わせる事ができます！
皆さんが普段使っている機械学習ライブラリでもOptunaが使えるかもしれませんので、ぜひ検討してみましょう。
今回はOptunaの以下の特徴について学びました。
- Lightweight, versatile, and platform agnostic architecture
- Pythonic search spaces

次のチュートリアルでは、Optunaの一歩進んだ使い方に関して学んでみましょう。
- Efficient optimization algorithms
- Easy parallelization
- Quick visualization

どうぞお楽しみに！